In [120]:
import time
from typing import List

import pandas as pd
from dataclasses import dataclass
from platform import platform

if "Windows" in platform():
    from pywinauto.keyboard import send_keys


@dataclass
class Entry:
    index: str = ""
    code: str = ""
    description: str = ""
    location: str = ""
    quantity: str = ""
    cost: str = ""
    price: str = ""
    notes: str = ""

    def __getitem__(self, key):
        return getattr(self, key)
    
    def __setitem__(self, key, value):
        setattr(self, key, value)

attrs = {
    "index": '#',
    "code": "Stock#",
    "description": "Description",
    "cost": "Cost",
}

# So here's how the bot is going to need to work
# Step 1: Read the Excel sheet
# Do this by opening every page in the spreadsheet using pandas
# and then read row by row to assemble a list of entries

entries: List[Entry] = []
dfs = pd.read_excel("cougarbot_data/stock.xls", sheet_name=None)
for sheet_name, df in dfs.items():
    df = df.astype(str)
    titles = list(df.iloc[0])
    indices = {
        attr: titles.index(value) + 1 for attr, value in attrs.items()
    }

    indices["price"] = titles.index("Cost") + 3
    qindex = titles.index("Abdn") + 1
    for row in df.itertuples():
        if row[indices["index"]] not in ["nan", '#']:
            if row[qindex + 1] == "nan" or row[qindex] == 'X':
                entry = Entry()
                for attr, index in indices.items():
                    entry[attr] = row[index]

                q = row[qindex] if row[qindex].isnumeric() else '1'
                entry["quantity"] = q
                entry["location"] = "ABDN"
                entries.append(entry)

            if row[qindex + 1] != "nan":
                entry = Entry()
                for attr, index in indices.items():
                    entry[attr] = row[index]

                q = '1' if row[qindex + 1] == 'X' else row[qindex + 1]
                entry["quantity"] = q
                entry["location"] = "HBY"
                entries.append(entry)

In [123]:
with open("cougarbot_data/signs.txt", 'r') as f:
    signs = f.read().split("\n\n")

x = {}
for s in signs:
    i, t = s.splitlines()
    x[i.strip('.')] = t.split("   ")[0].strip()